# Create Text Docs

In [1]:
from scraping import *
from pathlib import Path
import pandas as pd
import textwrap

In [2]:
p = Path.cwd()
path_parent = p.parents[0]

In [3]:
# database
path_db = str(path_parent / "database" / "youbemom-merged.db")
# topics
path_topic_percents = str(path_parent / "clean_data" / "topics_{0}_{1}_{2}.csv")

In [4]:
def gen_cols(n):
    return ["topic_{}".format(str(i).zfill(2)) for i in range(n)]

In [5]:
def write_line(fn, line):
    with open(fn, 'a') as f:
        f.write(line)

In [6]:
def wrap_line(line, indent_n):
    indent = "  " * indent_n
    wrapper = textwrap.TextWrapper(width=70)
    filled = wrapper.fill(text=line)
    return textwrap.indent(text=filled, prefix=indent)

In [7]:
def exp_ceil(n):
    exp = 1
    while n >= 10**exp:
        exp += 1
    return exp

In [8]:
def write_wrapped(sf, col, rank, line, blanks):
    path_text_files = str(path_parent / "clean_data" / "text_posts" / sf / "post_{0}_{1}_{2}.txt")
    spaces = "\n" * blanks
    write_line(path_text_files.format(sf, col, rank), line + spaces)

In [9]:
def make_topn_text(topn, sf, group, n):
    sql = "SELECT family_id, message_id, parent_id, title, body FROM posts WHERE family_id={}"
    df = pd.read_csv(path_topic_percents.format(sf, group, n))
    cols = gen_cols(n)
    for col in cols:
        df = df.sort_values(by=col, ascending=False)
        for i in range(topn):
            rank = str(i).zfill(exp_ceil(max(range(topn))))
            current_parent = []
            indent_n = 0
            family_id = df.iloc[i]['family_id']
            score = df.iloc[i][col]
            write_wrapped(sf, col, rank, "subforum:  {}".format(sf), 1)
            write_wrapped(sf, col, rank, "topic:     {}".format(col), 1)
            write_wrapped(sf, col, rank, "rank:      {}".format(rank), 1)
            write_wrapped(sf, col, rank, "family_id: {}".format(family_id), 1)
            write_wrapped(sf, col, rank, "score:     {}".format(score), 2)
            cur.execute(sql.format(family_id))
            rows = cur.fetchall()
            for row in rows:
                message_id = row[1]
                parent_id = row[2]
                title = row[3]
                body = row[4]
                if current_parent == []:
                    line = wrap_line(title + " " + body, indent_n)
                    write_wrapped(sf, col, rank, line, 2)
                    current_parent.insert(0, message_id)
                else:
                    while current_parent[0] != str(parent_id):
                        indent_n = indent_n - 1
                        current_parent.pop(0)
                    indent_n = indent_n + 1
                    line = wrap_line(title + " " + body, indent_n)
                    write_wrapped(sf, col, rank, line, 2)
                    current_parent.insert(0, message_id)

In [10]:
conn = create_connection(path_db)
cur = conn.cursor()

In [11]:
topn = 100

In [12]:
sf = "special-needs"
group = "all"
n = 20
make_topn_text(topn, sf, group, n)

In [13]:
sf = "toddler"
group = "all"
n = 15
make_topn_text(topn, sf, group, n)